# PyFlink 安装

### 作者：胖胖揽住

### 版本 2023.11.15

## Anaconda3 安装

首先从TUNA下载Anaconda3安装包。

```Bash
wget https://mirrors.tuna.tsinghua.edu.cn/anaconda/archive/Anaconda3-2023.09-0-Linux-x86_64.sh
sh Anaconda3-2023.09-0-Linux-x86_64.sh
```
安装过程中，请使用默认设置。
应该安装在`~/anaconda3`。

## Python 3.9 安装

通过 conda 安装 Python 3.9 将变得简单可靠。

```Bash
conda create -n pyflink_39 python=3.9
conda activate pyflink_39
```


## Apache-Flink 安装

先去[Apache 官网](https://dlcdn.apache.org/flink/)下载安装 flink，这里以 1.18.0 为例：

```Bash
wget https://dlcdn.apache.org/flink/flink-1.18.0/flink-1.18.0-bin-scala_2.12.tgz
sudo tar -zxvf flink-1.18.0-bin-scala_2.12.tgz  -C /usr/local   
```

修改目录名称，并设置权限，命令如下：
```Bash
cd /usr/local
sudo mv / flink-1.18.0 ./flink #这里是因为我这里下的是这个版本，读者需要酌情调整
sudo chown -R hadoop:hadoop ./flink ##这里是因为我这里虚拟机的用户名是这个，读者需要酌情调整
```

Flink解压缩并且设置好权限后，直接就可以在本地模式运行，不需要修改任何配置。
如果要做调整，可以编辑`“/usr/local/flink/conf/flink-conf.yam`这个文件。
比如其中的`env.java.home`参就可以设置为本地Java的绝对路径
不过一般不需要手动修改什么配置。

不过，需要注意的是，Flink现在需要的是Java11，所以需要用下列命令手动安装一下：
```Bash
sudo apt install openjdk-11-jdk -y
```

接下来还需要修接下来还需要修改配置文件，添加环境变量：

```Bash
nano ~/.bashrc
```

文件中添加如下内容：
```
export FLINK_HOME=/usr/local/flink
export PATH=$FLINK_HOME/bin:$PATH
```

保存并退出.bashrc文件，然后执行如下命令让配置文件生效：
```Bash
source ~/.bashrc
```

## 安装 Python 依赖包

然后使用 pip 安装 apache-flink 包， 以及 Kafka-python 等等依赖包

```Bash
pip install apache-flink 
pip install kafka-python chardet pandas numpy scipy simpy 
pip install matplotlib cython sympy xlrd pyopengl BeautifulSoup4 pyqt6 scikit-learn requests tensorflow torch keras tqdm gym DRL
```

## 代码说明

本文代码修改自官方[文档版本1.18](https://nightlies.apache.org/flink/flink-docs-release-1.18/docs/dev/python/datastream_tutorial/)。

In [1]:
# 基本操作：Map, Filter, Keyby

import json
import logging
import sys
from pyflink.common import Types
from pyflink.datastream import StreamExecutionEnvironment

# 定义show函数，用于显示数据流
def show(ds, env):
    ds.print()
    env.execute()

# 定义update_tel函数，用于更新tel字段
def update_tel(data):
    json_data = json.loads(data.info)
    json_data['tel'] += 1
    return data.id, json.dumps(json_data)

# 定义filter_by_id函数，用于过滤id字段
def filter_by_id(data):
    return data.id == 1

# 定义map_country_tel函数，用于将国家字段和tel字段映射到元组中
def map_country_tel(data):
    json_data = json.loads(data.info)
    return json_data['addr']['country'], json_data['tel']

# 定义key_by_country函数，用于将元组中的国家字段作为key
def key_by_country(data):
    return data[0]

if __name__ == '__main__':
    logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(message)s")
    env = StreamExecutionEnvironment.get_execution_environment()
    env.set_parallelism(1)
    ds = env.from_collection(
        collection=[
            (1, '{"name": "Flink", "tel": 111, "addr": {"country": "Germany", "city": "Berlin"}}'),
            (2, '{"name": "hello", "tel": 222, "addr": {"country": "China", "city": "Shanghai"}}'),
            (3, '{"name": "world", "tel": 333, "addr": {"country": "USA", "city": "NewYork"}}'),
            (4, '{"name": "PyFlink", "tel": 444, "addr": {"country": "China", "city": "Hangzhou"}}')
        ],
        type_info=Types.ROW_NAMED(["id", "info"], [Types.INT(), Types.STRING()])
    )
    print('\nFirst we map it: \n')
    # 调用show函数，显示数据流
    show(ds.map(update_tel), env)
    
    print('\nThen we filter it: \n')
    # 调用show函数，显示过滤后的数据流
    show(ds.filter(filter_by_id), env)

    print('\nThen we select it: \n')
    # 调用show函数，显示按照国家字段分组后的数据流
    show(ds.map(map_country_tel).key_by(key_by_country), env)

Using Any for unsupported type: typing.Sequence[~T]
No module named google.cloud.bigquery_storage_v1. As a result, the ReadFromBigQuery transform *CANNOT* be used with `method=DIRECT_READ`.

First we map it: 

(1, '{"name": "Flink", "tel": 112, "addr": {"country": "Germany", "city": "Berlin"}}')
(2, '{"name": "hello", "tel": 223, "addr": {"country": "China", "city": "Shanghai"}}')
(3, '{"name": "world", "tel": 334, "addr": {"country": "USA", "city": "NewYork"}}')
(4, '{"name": "PyFlink", "tel": 445, "addr": {"country": "China", "city": "Hangzhou"}}')

Then we filter it: 

+I[1,{"name": "Flink", "tel": 111, "addr": {"country": "Germany", "city": "Berlin"}}]

Then we select it: 

('Germany', 111)
('China', 222)
('USA', 333)
('China', 444)


In [2]:
# 处理 Json 数据


import json
import logging
import sys

from pyflink.datastream import StreamExecutionEnvironment


def process_json_data():
    # 获取执行环境
    env = StreamExecutionEnvironment.get_execution_environment()

    # define the source
    # 定义源数据
    ds = env.from_collection(
        collection=[
            (1, '{"name": "Flink", "tel": 111, "addr": {"country": "Germany", "city": "Berlin"}}'),
            (2, '{"name": "hello", "tel": 222, "addr": {"country": "China", "city": "Shanghai"}}'),
            (3, '{"name": "world", "tel": 333, "addr": {"country": "USA", "city": "NewYork"}}'),
            (4, '{"name": "PyFlink", "tel": 444, "addr": {"country": "China", "city": "Hangzhou"}}')]
    )

    # 定义更新电话号码的函数
    def update_tel(data):
        # parse the json
        # 解析json数据
        json_data = json.loads(data[1])
        # 更新电话号码
        json_data['tel'] += 1
        # 返回更新后的数据
        return data[0], json_data

    # 定义过滤函数，过滤掉国家不是中国的数据
    def filter_by_country(data):
        # the json data could be accessed directly, there is no need to parse it again using
        # json.loads
        # 直接访问json数据，不需要使用json.loads
        return "China" in data[1]['addr']['country']

    # 调用map函数，更新电话号码，并过滤掉国家不是中国的数据
    ds.map(update_tel).filter(filter_by_country).print()

    # submit for execution
    # 提交执行
    env.execute()


if __name__ == '__main__':
    # 设置日志输出格式
    logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(message)s")

    # 调用process_json_data函数
    process_json_data()

16> (2, {'name': 'hello', 'tel': 223, 'addr': {'country': 'China', 'city': 'Shanghai'}})
2> (4, {'name': 'PyFlink', 'tel': 445, 'addr': {'country': 'China', 'city': 'Hangzhou'}})


In [3]:
# 状态读取

from pyflink.common import Time
from pyflink.common.typeinfo import Types
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.functions import KeyedProcessFunction, RuntimeContext
from pyflink.datastream.state import ValueStateDescriptor, StateTtlConfig


# 定义一个Sum类，继承自KeyedProcessFunction
class Sum(KeyedProcessFunction):

    # 初始化函数
    def __init__(self):
        self.state = None

    # 打开函数，获取运行时上下文
    def open(self, runtime_context: RuntimeContext):
        # 创建一个状态描述符，类型为float
        state_descriptor = ValueStateDescriptor("state", Types.FLOAT())
        # 创建一个状态TTL配置，设置TTL时间为1秒，更新类型为OnReadAndWrite，禁用后台清理
        state_ttl_config = StateTtlConfig \
            .new_builder(Time.seconds(1)) \
            .set_update_type(StateTtlConfig.UpdateType.OnReadAndWrite) \
            .disable_cleanup_in_background() \
            .build()
        # 启用TTL，并传入TTL配置
        state_descriptor.enable_time_to_live(state_ttl_config)
        # 获取状态
        self.state = runtime_context.get_state(state_descriptor)

    # 处理元素函数
    def process_element(self, value, ctx: 'KeyedProcessFunction.Context'):
        # retrieve the current count
        # 获取当前状态
        current = self.state.value()
        # 如果当前状态为空，则设置为0
        if current is None:
            current = 0

        # update the state's count
        # 更新状态的计数
        current += value[1]
        # 更新状态
        self.state.update(current)

        # 返回元组
        yield value[0], current


# 定义一个state_access_demo函数，用于演示状态访问
def state_access_demo():
    # 获取运行时环境
    env = StreamExecutionEnvironment.get_execution_environment()

    # 从集合中创建一个流
    ds = env.from_collection(
        collection=[
            ('Alice', 110.1),
            ('Bob', 30.2),
            ('Alice', 20.0),
            ('Bob', 53.1),
            ('Alice', 13.1),
            ('Bob', 3.1),
            ('Bob', 16.1),
            ('Alice', 20.1)
        ],
        type_info=Types.TUPLE([Types.STRING(), Types.FLOAT()]))

    # apply the process function onto a keyed stream
    # 应用处理函数，对流中的每一个元素进行处理
    ds.key_by(lambda value: value[0]) \
      .process(Sum()) \
      .print()

    # submit for execution
    # 提交执行
    env.execute()


# 调用state_access_demo函数
if __name__ == '__main__':
    state_access_demo()

7> ('Alice', 110.0999984741211)
7> ('Alice', 130.0999984741211)
7> ('Alice', 143.19999885559082)
7> ('Alice', 163.29999923706055)
2> ('Bob', 30.200000762939453)
2> ('Bob', 83.29999923706055)
2> ('Bob', 86.39999914169312)
2> ('Bob', 102.49999952316284)


In [4]:
# 事件时间


from pyflink.common import Time, WatermarkStrategy, Duration
from pyflink.common.typeinfo import Types
from pyflink.common.watermark_strategy import TimestampAssigner
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.functions import KeyedProcessFunction, RuntimeContext
from pyflink.datastream.state import ValueStateDescriptor, StateTtlConfig


# 定义Sum类，继承自KeyedProcessFunction
class Sum(KeyedProcessFunction):

    # 初始化函数
    def __init__(self):
        self.state = None

    # 打开函数，获取状态描述符，设置状态TTL配置，并设置状态描述符的TTL配置
    def open(self, runtime_context: RuntimeContext):
        state_descriptor = ValueStateDescriptor("state", Types.FLOAT())
        state_ttl_config = StateTtlConfig \
            .new_builder(Time.seconds(1)) \
            .set_update_type(StateTtlConfig.UpdateType.OnReadAndWrite) \
            .disable_cleanup_in_background() \
            .build()
        state_descriptor.enable_time_to_live(state_ttl_config)
        self.state = runtime_context.get_state(state_descriptor)

    # 处理元素函数，获取当前状态，更新状态，并注册一个2秒后的定时器
    def process_element(self, value, ctx: 'KeyedProcessFunction.Context'):
        # retrieve the current count
        current = self.state.value()
        if current is None:
            current = 0

        # update the state's count
        current += value[2]
        self.state.update(current)

        # register an event time timer 2 seconds later
        ctx.timer_service().register_event_time_timer(ctx.timestamp() + 2000)

    # 定时器函数，获取当前状态，并输出
    def on_timer(self, timestamp: int, ctx: 'KeyedProcessFunction.OnTimerContext'):
        yield ctx.get_current_key(), self.state.value()


# 定义MyTimestampAssigner类，继承自TimestampAssigner
class MyTimestampAssigner(TimestampAssigner):

    # 提取时间戳函数，根据value和record_timestamp获取时间戳
    def extract_timestamp(self, value, record_timestamp: int) -> int:
        return int(value[0])


# 定义event_timer_timer_demo函数，获取执行环境，从集合中获取数据，设置时间戳和水位策略，并应用处理函数，提交执行
def event_timer_timer_demo():
    env = StreamExecutionEnvironment.get_execution_environment()

    ds = env.from_collection(
        collection=[
            (1000, 'Alice', 110.1),
            (4000, 'Bob', 30.2),
            (3000, 'Alice', 20.0),
            (2000, 'Bob', 53.1),
            (5000, 'Alice', 13.1),
            (3000, 'Bob', 3.1),
            (7000, 'Bob', 16.1),
            (10000, 'Alice', 20.1)
        ],
        type_info=Types.TUPLE([Types.LONG(), Types.STRING(), Types.FLOAT()]))

    ds = ds.assign_timestamps_and_watermarks(
        WatermarkStrategy.for_bounded_out_of_orderness(Duration.of_seconds(2))
                         .with_timestamp_assigner(MyTimestampAssigner()))

    # apply the process function onto a keyed stream
    ds.key_by(lambda value: value[1]) \
      .process(Sum()) \
      .print()

    # submit for execution
    env.execute()


if __name__ == '__main__':
    event_timer_timer_demo()

Exception in thread read_grpc_client_inputs:
Traceback (most recent call last):
  File "/home/fred/anaconda3/envs/pyflink_39/lib/python3.9/threading.py", line 980, in _bootstrap_inner
Exception in thread read_grpc_client_inputs:
Traceback (most recent call last):
  File "/home/fred/anaconda3/envs/pyflink_39/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/fred/anaconda3/envs/pyflink_39/lib/python3.9/threading.py", line 917, in run
    self.run()
  File "/home/fred/anaconda3/envs/pyflink_39/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/home/fred/anaconda3/envs/pyflink_39/lib/python3.9/site-packages/apache_beam/runners/worker/data_plane.py", line 669, in <lambda>
Exception in thread read_grpc_client_inputs:
Traceback (most recent call last):
  File "/home/fred/anaconda3/envs/pyflink_39/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    target=lambda: self._read_inputs(elements_iterat

7> ('Alice', 163.29999923706055)
7> ('Alice', 163.29999923706055)
7> ('Alice', 163.29999923706055)
7> ('Alice', 163.29999923706055)
2> ('Bob', 102.49999952316284)
2> ('Bob', 102.49999952316284)
2> ('Bob', 102.49999952316284)
2> ('Bob', 102.49999952316284)


# 上手流数据

## 流数据的来源

网站数据采集，用户行为产生，购物网站、社交网站。
传感器采集，物联网传输，科研观测、探测器。

## 从本地数据生成流数据

将本地文件读取，发送给Kafka，然后再从Kafka来读取。

## 使用Docker搭建本地Kafka集群

操作系统选择 Ubuntu 22.04.3   

1. 安装 Docker 和 Docker Compose:
```Bash
sudo apt install Docker Docker-compose
```
2. 创建本地 `docker-compose.yml` 文件，其中包含以下内容：

```yaml
version: '3'
services:
  zookeeper:
    image: 'bitnami/zookeeper:latest'
    environment:
      - ALLOW_ANONYMOUS_LOGIN=yes
  kafka:
    image: 'bitnami/kafka:latest'
    ports:
      - '9092:9092'
    environment:
      - KAFKA_ADVERTISED_HOST_NAME=localhost
      - KAFKA_ZOOKEEPER_CONNECT=zookeeper:2181
      - KAFKA_ADVERTISED_LISTENERS=PLAINTEXT://localhost:9092
      - KAFKA_LISTENERS=PLAINTEXT://0.0.0.0:9092
      - KAFKA_CREATE_TOPICS=test:1:1
      - ALLOW_PLAINTEXT_LISTENER=yes
    depends_on:
      - zookeeper
```

3. 找到“docker-compose.yml”所在目录并运行以下命令：

````Bash
docker-compose up -d
````

这将运行一个包含 Zookeeper 实例和 Kafka 实例的本地 Kafka 集群，该集群将在本地主机的端口 9092 上运行。

In [6]:
# 使用 kafka-python 生成流的简单方法

# 以下代码使用kafka-python模块将数据发送到本地Kafka集群。
# 此代码打开一个名为 `hamlet.txt` 的文本文件，并将其内容作为流发送到指定的 Kafka 主题 `hamlet`：

# 导入KafkaProducer模块
from kafka import KafkaProducer
# 导入time模块
import time
# 导入os模块
import os

# 定义一个函数，用于将文件发送到Kafka
def send_file_to_kafka(file_path: str, topic: str, bootstrap_servers: str):
    # 创建一个KafkaProducer实例，用于发送消息
    producer = KafkaProducer(bootstrap_servers=bootstrap_servers)
    # 获取文件大小
    file_size = os.path.getsize(file_path)
    # 循环发送文件
    while True:
        # 打开文件
        with open(file_path, "rb") as f:
            # 循环读取文件
            while True:
                # 读取文件内容
                data = f.read(1024)
                # 如果没有内容，则跳出循环
                if not data:
                    break
                # 将文件内容发送到Kafka
                producer.send(topic, data)
                # 计算发送的字节数
                bytes_sent = len(data)
                # 打印发送的字节数
                print(f"Sent {bytes_sent} bytes to Kafka topic {topic}")
                # 计算发送的百分比
                percent_sent = (f.tell() / file_size) * 100
                # 打印发送的百分比
                print(f"{percent_sent:.2f}% of the file sent")
                # 等待3秒
                time.sleep(3)
        # 获取用户输入
        user_input = input("Press 'c' to continue sending the file or 'q' to quit: ")
        # 如果用户输入q，则退出循环
        if user_input == "q":
            break
# 调用函数，将hamlet.txt文件发送到Kafka的hamlet主题
send_file_to_kafka("./hamlet.txt",  "hamlet", "localhost:9092")
# 在此代码中，send_file_to_kafka 函数接受三个参数：file_path、topic 和 bootstrap_servers。
# file_path是本地文件的路径，topic是数据要发送到的Kafka主题，bootstrap_servers是Kafka集群的地址。
# 该函数使用with语句打开文件，读取其内容，并将它们作为流数据发送到指定的Kafka主题。
# 发送过程中，打印出发送进度，并使用time.sleep方法暂停0.1秒来控制发送速率。

Sent 1024 bytes to Kafka topic hamlet
0.57% of the file sent
Sent 1024 bytes to Kafka topic hamlet
1.13% of the file sent
Sent 1024 bytes to Kafka topic hamlet
1.70% of the file sent
Sent 1024 bytes to Kafka topic hamlet
2.27% of the file sent
Sent 1024 bytes to Kafka topic hamlet
2.83% of the file sent
Sent 1024 bytes to Kafka topic hamlet
3.40% of the file sent
Sent 1024 bytes to Kafka topic hamlet
3.97% of the file sent
Sent 1024 bytes to Kafka topic hamlet
4.53% of the file sent
Sent 1024 bytes to Kafka topic hamlet
5.10% of the file sent
Sent 1024 bytes to Kafka topic hamlet
5.66% of the file sent
Sent 1024 bytes to Kafka topic hamlet
6.23% of the file sent
Sent 1024 bytes to Kafka topic hamlet
6.80% of the file sent
Sent 1024 bytes to Kafka topic hamlet
7.36% of the file sent
Sent 1024 bytes to Kafka topic hamlet
7.93% of the file sent
Sent 1024 bytes to Kafka topic hamlet
8.50% of the file sent


KeyboardInterrupt: 

In [7]:
# 使用 kafka-python 展现流数据的简单方法

from kafka import KafkaConsumer

# 创建一个KafkaConsumer实例，用于从Kafka主题中读取消息
consumer = KafkaConsumer(
    # 指定要读取的消息主题
    "hamlet",
    # 指定Kafka服务器的地址和端口
    bootstrap_servers=["localhost:9092"],
    # 指定当消费者重新启动时，它应该从哪个偏移量开始读取消息
    auto_offset_reset="earliest",
    # 指定是否在消费者处理消息时，应该提交偏移量
    enable_auto_commit=True,
    # 指定消费者组，用于提交偏移量
    group_id="my-group",
    # 指定消息的解码方式
    value_deserializer=lambda x: x.decode("utf-8")
)

# 循环读取Kafka主题中的消息，并打印消息长度和消息内容
for message in consumer:
    print(f"Received {len(message.value)} bytes from Kafka topic {message.topic}")
    print(f"{message.value}")

# 在上面的代码中，我们使用`KafkaConsumer`类来创建一个消费者对象。
# 我们将 `hamlet` 作为主题名称传递给构造函数。
# 我们还传递 `localhost:9092` 作为引导服务器的地址。
# 我们使用 `value_deserializer` 参数来解码从 Kafka 主题收到的消息。
# 我们使用 `for` 循环来迭代消费者对象，并使用 `print` 函数来打印消息的内容。

KeyboardInterrupt: 

In [1]:
import os
import argparse
import logging
import sys
import numpy as np 
import pandas as pd
from pyflink.table import StreamTableEnvironment
from pyflink.common import WatermarkStrategy, Encoder, Types
from pyflink.datastream import StreamExecutionEnvironment, RuntimeExecutionMode
from pyflink.datastream.connectors.file_system import FileSource, StreamFormat, FileSink, OutputFileConfig, RollingPolicy
from pyflink.common import Types, SimpleStringSchema
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.connectors.kafka import FlinkKafkaProducer, FlinkKafkaConsumer

# 定义split函数，用于将字符串按照空格分割
def split(line):
    # 使用yield from语法，将line按照空格分割，并返回分割后的字符串
    yield from line.split()

# 定义read_from_kafka函数，用于从Kafka中读取数据
def read_from_kafka():
    # 获取StreamExecutionEnvironment实例
    env = StreamExecutionEnvironment.get_execution_environment()    
    # 添加FlinkKafkaConsumer的jar包
    env.add_jars("file:///home/hadoop/Desktop/PyFlink-Tutorial/flink-sql-connector-kafka-3.1-SNAPSHOT.jar")
    # 打印信息
    print("start reading data from kafka")
    # 创建FlinkKafkaConsumer实例，用于从Kafka中读取数据
    kafka_consumer = FlinkKafkaConsumer(
        topics='hamlet', # The topic to consume messages from
        deserialization_schema= SimpleStringSchema('UTF-8'), # The schema to deserialize messages
        properties={'bootstrap.servers': 'localhost:9092', 'group.id': 'my-group'} # The Kafka broker address and consumer group ID
    )
    # 从最早的记录开始读取数据
    kafka_consumer.set_start_from_earliest()
    # 将FlinkKafkaConsumer实例添加到StreamExecutionEnvironment实例中，并打印读取到的数据
    env.add_source(kafka_consumer).print()
    # 执行StreamExecutionEnvironment实例
    env.execute()

# 调用read_from_kafka函数
if __name__ == '__main__':
    # 设置日志级别
    logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(message)s")
    # 调用read_from_kafka函数
    read_from_kafka()

start reading data from kafka


Py4JJavaError: An error occurred while calling o0.execute.
: org.apache.flink.runtime.client.JobExecutionException: Job execution failed.
	at org.apache.flink.runtime.jobmaster.JobResult.toJobExecutionResult(JobResult.java:144)
	at org.apache.flink.runtime.minicluster.MiniClusterJobClient.lambda$getJobExecutionResult$3(MiniClusterJobClient.java:141)
	at java.base/java.util.concurrent.CompletableFuture$UniApply.tryFire(CompletableFuture.java:646)
	at java.base/java.util.concurrent.CompletableFuture.postComplete(CompletableFuture.java:510)
	at java.base/java.util.concurrent.CompletableFuture.complete(CompletableFuture.java:2147)
	at org.apache.flink.runtime.rpc.pekko.PekkoInvocationHandler.lambda$invokeRpc$1(PekkoInvocationHandler.java:268)
	at java.base/java.util.concurrent.CompletableFuture.uniWhenComplete(CompletableFuture.java:863)
	at java.base/java.util.concurrent.CompletableFuture$UniWhenComplete.tryFire(CompletableFuture.java:841)
	at java.base/java.util.concurrent.CompletableFut

In [ ]:
# 简单的词频统计

# 导入os模块
import os
# 导入re模块
import re
# 导入Counter模块
from collections import Counter
# 导入StreamTableEnvironment模块
from pyflink.table import StreamTableEnvironment
# 导入StreamExecutionEnvironment模块
from pyflink.datastream import StreamExecutionEnvironment
# 导入FlinkKafkaConsumer模块
from pyflink.datastream.connectors.kafka import FlinkKafkaConsumer
# 导入SimpleStringSchema模块
from pyflink.common import SimpleStringSchema

# 定义去除标点符号的函数
def remove_punctuation(text):
    # 使用正则表达式去除标点符号
    return re.sub(r'[^\w\s]','',text)

# 定义统计单词的函数
def count_words(text):
    # 将文本按空格分割成单词列表
    words = text.split()
    # 使用Counter模块统计单词出现次数
    return Counter(words)

# 定义从Kafka读取数据的函数
def read_from_kafka():
    # 获取StreamExecutionEnvironment实例
    env = StreamExecutionEnvironment.get_execution_environment()    
    # 添加flink-sql-connector-kafka-3.1-SNAPSHOT.jar包
    env.add_jars("file:///home/hadoop/Desktop/PyFlink-Tutorial/flink-sql-connector-kafka-3.1-SNAPSHOT.jar")
    # 打印从Kafka读取数据的信息
    print("start reading data from kafka")
    # 创建FlinkKafkaConsumer实例
    kafka_consumer = FlinkKafkaConsumer(
        topics='hamlet', # The topic to consume messages from
        deserialization_schema= SimpleStringSchema('UTF-8'), # The schema to deserialize messages
        properties={'bootstrap.servers': 'localhost:9092', 'group.id': 'my-group'} # The Kafka broker address and consumer group ID
    )
    # 从最早的记录开始读取数据
    kafka_consumer.set_start_from_earliest()
    # 将FlinkKafkaConsumer实例添加到StreamExecutionEnvironment实例中
    stream = env.add_source(kafka_consumer)
    # 将StreamExecutionEnvironment实例中的数据映射为去除标点符号的文本
    stream_remove_punctuation = stream.map(lambda x: remove_punctuation(x))
    # 将去除标点符号的文本映射为统计单词的文本
    stream_count_words = stream_remove_punctuation.map(lambda x: count_words(x))
    # 打印统计单词的文本
    stream_count_words.print()
    # 执行StreamExecutionEnvironment实例
    env.execute()

# 调用read_from_kafka函数
read_from_kafka()

In [ ]:
# 更详细的词频统计

# 导入 argparse、io、json、logging、os、pandas、re、Counter、StringIO、FlinkKafkaConsumer、StreamExecutionEnvironment、DataTypes、EnvironmentSettings、FormatDescriptor、Schema、StreamTableEnvironment、TableEnvironment、udf 模块
import argparse
import io
import json
import logging
import os
import pandas as pd
import re
from collections import Counter
from io import StringIO
from pyflink.common import SimpleStringSchema, Time
from pyflink.datastream.connectors.kafka import FlinkKafkaConsumer
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import (DataTypes, EnvironmentSettings, FormatDescriptor,
                           Schema, StreamTableEnvironment, TableDescriptor,
                           TableEnvironment, udf)
from pyflink.table.expressions import col, lit

# 定义去除标点符号的函数
def remove_punctuation(text):
    return re.sub(r'[^\w\s]','',text)

# 定义计算字节数的函数
def count_bytes(text):
    return len(text.encode('utf-8'))

# 定义计算单词数量的函数
def count_words(text):
    words = text.split()
    result = dict(Counter(words))
    max_word = max(result, key=result.get)
    return {'total_bytes': count_bytes(text), 'total_words': len(words), 'most_frequent_word': max_word, 'most_frequent_word_count': result[max_word]}

# 定义从Kafka读取数据的函数
def read_from_kafka():
    # 获取StreamExecutionEnvironment实例
    env = StreamExecutionEnvironment.get_execution_environment()  
    # 添加flink-sql-connector-kafka-3.1-SNAPSHOT.jar包
    env.add_jars("file:///home/hadoop/Desktop/PyFlink-Tutorial/flink-sql-connector-kafka-3.1-SNAPSHOT.jar")
    print("start reading data from kafka")
    # 创建FlinkKafkaConsumer实例，指定主题、反序列化函数、配置参数
    kafka_consumer = FlinkKafkaConsumer(
        topics='hamlet', 
        deserialization_schema= SimpleStringSchema('UTF-8'), 
        properties={'bootstrap.servers': 'localhost:9092', 'group.id': 'my-group'} 
    )
    # 从最早的日志开始读取
    kafka_consumer.set_start_from_earliest()
    # 将Kafka日志流转换为流表
    stream_original_text = env.add_source(kafka_consumer)
    # 对流表中的每一行进行去除标点符号操作
    stream_remove_punctuation = stream_original_text.map(lambda x: remove_punctuation(x))
    # 对流表中的每一行进行计算单词数量的操作
    stream_count_words = stream_remove_punctuation.map(lambda x: count_words(x))
    # 将流表中的每一行打印出来
    stream_count_words.print()
    # 执行流计算
    env.execute()
read_from_kafka()

# 玩转 CSV

假设我们得到一个“data.csv”文件，其中包含任何内容，并且该文件中只有年份数据才是我们需要的。
我们首先使用以下代码生成“StreamGeneratorCSV”，将“CSV”文件转换为“Kafka Stream”。

In [ ]:
# 一个简单的 CSV 流生成器

#以下代码使用kafka-python模块将数据发送到本地Kafka集群。
#此代码打开一个名为“hamlet.txt”的文本文件，并将其内容作为流发送到指定的 Kafka 主题“hamlet”：

from kafka import KafkaProducer
import time
import os
import chardet

def send_file_to_kafka(file_path: str, topic: str, bootstrap_servers: str):
    '''
    Send a file to a Kafka topic
    :param file_path: path to the local file
    :param topic: Kafka topic to which the data should be sent
    :param bootstrap_servers: address of the Kafka cluster
    '''
    # 创建一个KafkaProducer实例
    producer = KafkaProducer(bootstrap_servers=bootstrap_servers)
    # 获取文件大小
    file_size = os.path.getsize(file_path)

    # 检测文件编码
    with open(file_path, "rb") as f:
        result = chardet.detect(f.read())
        encoding = result["encoding"]

    # 获取文件行数
    with open(file_path, "r", encoding=encoding) as f:
        lines_total = len(f.readlines())

    lines_send = 0
    while True:
        # 打开文件
        with open(file_path, "rb") as f:
            while True:
                # 读取文件10行
                data = f.readlines(10)
                if not data:
                    break
                # 将数据转换为字符串
                data_str = str(data)
                # 将字符串转换为字节
                data_bytes = data_str.encode()
                # 将字节发送到Kafka
                producer.send(topic, data_bytes)
                # 记录发送的行数
                lines_send += 10
                # 计算已发送的百分比
                percent_sent = (lines_send / lines_total) * 100                
                # 计算已发送的字节数
                bytes_sent = len(data_bytes)
                print(f"Sent {bytes_sent} bytes {topic} {percent_sent:.2f}% sent")
                # 每3秒检查一次
                time.sleep(3)
                
        # 询问是否继续发送
        user_input = input("Press 'c' to continue sending the file or 'q' to quit: ")
        if user_input == "q":
            break
# 调用send_file_to_kafka函数，将文件data.csv发送到Kafka主题data，Kafka集群的地址为localhost:9092
send_file_to_kafka("./data.csv",  "data", "localhost:9092")

# 解释以上代码
# 在这个代码中，send_file_to_kafka 函数接受三个参数：file_path、topic 和 bootstrap_servers。
# file_path 是本地文件的路径，topic 是要将数据发送到的 Kafka 主题，bootstrap_servers 是 Kafka 集群的地址。
# 该函数使用 with 语句打开文件，读取其内容，并将其作为流数据发送到指定的 Kafka 主题。在发送过程中，它会打印出传输进度，并使用 time.sleep 方法暂停 3 秒以控制发送速率。



# 输出年份数值

StreamShowerWithFlinkCSV.py 是一个使用 DataStream 处理 CSV 文件的 Python 脚本。实际上，下面的代码使用 re 函数。
但这不重要，只是对从 CSV 文件生成的 DataStream 随便试试。

In [ ]:
# StreamShowerWithFlinkCSV.py

# 导入正则表达式模块、参数解析模块、日志模块、系统模块、numpy模块、pandas模块、pyflink模块
import re
import argparse
import logging
import sys
import numpy as np 
import pandas as pd
from pyflink.table import StreamTableEnvironment
from pyflink.common import WatermarkStrategy, Encoder, Types
from pyflink.datastream import StreamExecutionEnvironment, RuntimeExecutionMode
from pyflink.datastream.connectors.file_system import FileSource, StreamFormat, FileSink, OutputFileConfig, RollingPolicy
from pyflink.common import Types, SimpleStringSchema
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.connectors.kafka import FlinkKafkaProducer, FlinkKafkaConsumer

# 定义split函数，用于将字符串拆分成单个单词
def split(line):
    yield from line.split()

# 定义read_from_kafka函数，用于从Kafka读取数据
def read_from_kafka():
    # 定义Kafka消费的起始年份
    Year_Begin =1999
    # 定义Kafka消费的结束年份
    Year_End = 2023
    # 获取StreamExecutionEnvironment实例
    env = StreamExecutionEnvironment.get_execution_environment()    
    # 添加jars包
    env.add_jars("file:///home/hadoop/Desktop/PyFlink-Tutorial/flink-sql-connector-kafka-3.1-SNAPSHOT.jar")
    # 打印开始读取Kafka数据
    print("start reading data from kafka")

    # 创建Kafka消费者，用于从Kafka读取数据
    kafka_consumer = FlinkKafkaConsumer(
        topics='data', # The topic to consume messages from
        deserialization_schema= SimpleStringSchema('UTF-8'), # The schema to deserialize messages
        properties={'bootstrap.servers': 'localhost:9092', 'group.id': 'my-group'} # The Kafka broker address and consumer group ID
    )

    # 从最早的偏移量开始读取Kafka数据
    kafka_consumer.set_start_from_earliest()

    # 添加Kafka消费者，并过滤掉不在指定年份范围内的数据
    env.add_source(kafka_consumer).map(lambda x: ' '.join(re.findall(r'\d+', x))).filter(lambda x: any([Year_Begin <= int(i) <= Year_End for i in x.split()])).map(lambda x:  [i for i in x.split() if Year_Begin <= int(i) <= Year_End][0]).print()
    # 执行StreamExecutionEnvironment
    env.execute()

# 调用read_from_kafka函数
if __name__ == '__main__':
    read_from_kafka()

`MapFunction`: 将一个元素作为输入并将一个元素作为输出的函数。通过对每个元素应用转换，它可用于转换数据流。
`FlatMapFunction`：将一个元素作为输入，并将零个、一个或多个元素作为输出的函数。它可通过对每个元素应用变换来转换数据流。
`FilterFunction`: 将一个元素作为输入并返回一个布尔值的函数。它可用于删除不符合特定条件的元素，从而过滤数据流。
`KeySelector`: 从元素中提取键的函数。它可用于按键对数据流中的元素进行分组。
`ReduceFunction`: 还原函数 将两个元素作为输入并将一个元素作为输出的函数。它可以通过组合共享一个共同键的元素来聚合数据流。
`WindowFunction`: 将元素窗口作为输入并将一个或多个元素作为输出的函数。它可用于在数据流上定义窗口，并对每个窗口内的元素进行转换。